# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

# <a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** CNN = I used the resnet50 default (it's a great nn), but added in kaiming normal init for the fc layer, and added in batchnorm2d at the end of the convolution layers, and a bn1d before passing back.  
RNN - I used a 1 layer lstm.  I did some work with trying to init the hidden buffer (per the pytorch tutorial on lstm) but it didn't seem to really matter and dropped it. 
For the params, I used the threshold for words of 5 and 512 per the referenced paper (first link) and batch size of 10 at first to train faster and then 64 per the paper. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I left it as-is.  Resnet trained params were trained on 224 size (as I recall), and there was some data augmentation already there via the flip param...so felt it was a solid start.  It trained well so didn't revisit.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** For the CNN, only the classification (fully connected or linear) layer and batchnorm were set to trainable.  The main convolution 'body' was left exactly as is.  This is a good split b/c resnet was already highly trained to recognize objects/items so unlikely we are going to improve the feature filters - we just need to repurpose it from classification of 1000 as it's output, to building up a 512 feature vector for the LSTM instead and thus have a trainable head or linear layer.  
For the RNN, all the parameters were set to trainable as it is learning from scratch and so needs to be able to fully learn/train.  

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I used Adam - I just read a paper showing how Adam was the most robust optimizer across a variety of nn testing, so it was an easy choice.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
!pip install nltk
import nltk
nltk.download('punkt')

%matplotlib inline

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 400             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()




You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.08s)
creating index...


  0%|          | 981/414113 [00:00<01:25, 4826.45it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:10<00:00, 5907.93it/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 40846894.42it/s]


hidden_size param : 512


In [4]:
# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.linear.parameters())# +  #+ 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr= .003)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

In [17]:
batch_size=64

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr= .006)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

In [21]:
epoch=2
torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

In [19]:
num_epochs=2

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [20]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})
#total_step=20
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
        print("models saved")

# Close the training log file.
f.close()

in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1/41412], Loss: 3.5402, Perplexity: 34.4730in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [2/41412], Loss: 3.9188, Perplexity: 50.3375in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [3/41412], Loss: 3.4428, Perplexity: 31.2751in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [4/41412], Loss: 2.9591, Perplexity: 19.2811in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [5/41412], Loss: 3.4591, Perplexity: 31.7890in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [6/41412], Loss: 3.5793, Perplexity: 35.8502in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [7/41412], Loss: 3.2485, Perplexity: 25.7510in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [8/41412], Loss: 3.0424, Perplexity: 20.9551in forward - features tensor is  <class 'torch.Tensor'>


Epoch [1/2], Step [137/41412], Loss: 3.6671, Perplexity: 39.1366in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [138/41412], Loss: 2.9480, Perplexity: 19.0682in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [139/41412], Loss: 3.2605, Perplexity: 26.0625in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [140/41412], Loss: 3.4150, Perplexity: 30.4167in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [141/41412], Loss: 3.3657, Perplexity: 28.9532in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [142/41412], Loss: 3.2527, Perplexity: 25.8605in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [143/41412], Loss: 3.0951, Perplexity: 22.0905in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [144/41412], Loss: 2.8494, Perplexity: 17.2779in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [145/41412], Loss: 3.2

Epoch [1/2], Step [273/41412], Loss: 3.6169, Perplexity: 37.2231in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [274/41412], Loss: 3.8301, Perplexity: 46.0683in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [275/41412], Loss: 3.1749, Perplexity: 23.9237in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [276/41412], Loss: 2.5224, Perplexity: 12.4579in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [277/41412], Loss: 3.3027, Perplexity: 27.1855in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [278/41412], Loss: 2.6455, Perplexity: 14.0911in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [279/41412], Loss: 3.1467, Perplexity: 23.2601in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [280/41412], Loss: 3.3384, Perplexity: 28.1740in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [281/41412], Loss: 3.7

Epoch [1/2], Step [409/41412], Loss: 3.4485, Perplexity: 31.4534in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [410/41412], Loss: 3.1457, Perplexity: 23.2353in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [411/41412], Loss: 2.6257, Perplexity: 13.8148in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [412/41412], Loss: 2.7547, Perplexity: 15.7163in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [413/41412], Loss: 3.1727, Perplexity: 23.8723in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [414/41412], Loss: 3.3889, Perplexity: 29.6321in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [415/41412], Loss: 2.9400, Perplexity: 18.9157in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [416/41412], Loss: 3.0037, Perplexity: 20.1591in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [417/41412], Loss: 3.6

Epoch [1/2], Step [545/41412], Loss: 2.9297, Perplexity: 18.7223in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [546/41412], Loss: 3.5127, Perplexity: 33.5380in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [547/41412], Loss: 3.4562, Perplexity: 31.6973in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [548/41412], Loss: 2.4779, Perplexity: 11.9158in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [549/41412], Loss: 3.5384, Perplexity: 34.4127in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [550/41412], Loss: 2.5252, Perplexity: 12.4939in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [551/41412], Loss: 3.4584, Perplexity: 31.7657in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [552/41412], Loss: 2.5346, Perplexity: 12.6109in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [553/41412], Loss: 2.8

...

Epoch [1/2], Step [817/41412], Loss: 2.8432, Perplexity: 17.1710in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [818/41412], Loss: 3.1172, Perplexity: 22.5828in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [819/41412], Loss: 3.4867, Perplexity: 32.6795in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [820/41412], Loss: 2.8093, Perplexity: 16.5983in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [821/41412], Loss: 2.6021, Perplexity: 13.4925in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [822/41412], Loss: 3.1873, Perplexity: 24.2233in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [823/41412], Loss: 4.1853, Perplexity: 65.7158in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [824/41412], Loss: 3.3637, Perplexity: 28.8955in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [825/41412], Loss: 3.1

Epoch [1/2], Step [953/41412], Loss: 3.2784, Perplexity: 26.5339in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [954/41412], Loss: 2.9480, Perplexity: 19.0679in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [955/41412], Loss: 2.6604, Perplexity: 14.3024in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [956/41412], Loss: 3.3172, Perplexity: 27.5821in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [957/41412], Loss: 2.4357, Perplexity: 11.4240in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [958/41412], Loss: 2.9914, Perplexity: 19.9141in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [959/41412], Loss: 2.9110, Perplexity: 18.3748in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [960/41412], Loss: 3.9153, Perplexity: 50.1618in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [961/41412], Loss: 3.2

Epoch [1/2], Step [1089/41412], Loss: 3.3509, Perplexity: 28.5283in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1090/41412], Loss: 3.5833, Perplexity: 35.9912in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1091/41412], Loss: 4.0457, Perplexity: 57.1506in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1092/41412], Loss: 3.1629, Perplexity: 23.6388in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1093/41412], Loss: 2.7077, Perplexity: 14.9952in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1094/41412], Loss: 2.9025, Perplexity: 18.2192in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1095/41412], Loss: 3.0221, Perplexity: 20.5337in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1096/41412], Loss: 2.7021, Perplexity: 14.9110in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1097/41412], 

Epoch [1/2], Step [1157/41412], Loss: 2.5668, Perplexity: 13.0236in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1158/41412], Loss: 3.2586, Perplexity: 26.0133in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1159/41412], Loss: 2.6490, Perplexity: 14.1401in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1160/41412], Loss: 3.8278, Perplexity: 45.9633in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1161/41412], Loss: 2.3666, Perplexity: 10.6615in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1162/41412], Loss: 2.8354, Perplexity: 17.0370in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1163/41412], Loss: 2.9564, Perplexity: 19.2279in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1164/41412], Loss: 2.7321, Perplexity: 15.3645in forward - features tensor is  <class 'torch.Tensor'>
Epoch [1/2], Step [1165/41412], 

KeyboardInterrupt: 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.